In [ ]:
import tensorflow as tf
import numpy as np
from functools import reduce
from collections import Counter

In [ ]:
def get_data(train, test):
  with open(train) as f:
    sentences_train = f.read().split()
    # f.read().split('.\n'): split by new line
    # f.read().split(): split by space
  with open(test) as g:
    sentences_test = g.read().split()

  """
  for s in sentences_train[:10]: 
    print(s)
  print()
  for s in sentences_test[:10]: 
    print(s)
  """

  train_all = ""
  for i in sentences_train:
    train_all += " " + i
  train_all = train_all[1:]

  test_all = ""
  for i in sentences_test:
    test_all += " " + i
  test_all = test_all[1:]

  """
  print(train_all[:20])
  print()
  print(test_all[:20])
  print()
  for i in train_all[:100].split():
    print(i)
  print()
  """

  vocab = Counter(train_all.split())
  # Counter() counts the instances of each element in your list 
  # print(vocab)
  # print()

  convert_list = list(vocab)
  word2id = {w:i for i,w in enumerate(convert_list)}
  # print(word2id)
  # print()

  train2id = []
  for i in train_all.split():
    train2id.append(word2id[i])
  # print(train2id[:100])
  # print()

  test2id = []
  for i in test_all.split():
    test2id.append(word2id[i])
  # print(test2id[:100])


  return train2id, test2id, word2id


In [ ]:
get_data("train.txt", "test.txt")

([0,
  1,
  2,
  3,
  0,
  1,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  3,
  7,
  4,
  13,
  14,
  3,
  15,
  16,
  17,
  18,
  19,
  10,
  20,
  13,
  14,
  3,
  21,
  7,
  19,
  8,
  3,
  22,
  12,
  23,
  4,
  24,
  25,
  26,
  3,
  27,
  28,
  29,
  9,
  30,
  19,
  31,
  3,
  32,
  33,
  12,
  34,
  35,
  2,
  36,
  36,
  1,
  36,
  36,
  37,
  35,
  38,
  5,
  39,
  37,
  40,
  41,
  42,
  43,
  14,
  3,
  44,
  45,
  12,
  46,
  19,
  2,
  47,
  48,
  2,
  49,
  50,
  2,
  34,
  1,
  51,
  52,
  2,
  5,
  1,
  52,
  4,
  5,
  53,
  52,
  28,
  54,
  55,
  56,
  57,
  19,
  58,
  54,
  55,
  56,
  8,
  59,
  60,
  10,
  61,
  62,
  41,
  5,
  63,
  12,
  16,
  14,
  64,
  65,
  5,
  66,
  67,
  14,
  68,
  19,
  10,
  69,
  3,
  16,
  70,
  13,
  19,
  71,
  2,
  34,
  72,
  19,
  73,
  74,
  2,
  72,
  19,
  73,
  74,
  71,
  72,
  19,
  58,
  71,
  75,
  25,
  73,
  5,
  6,
  76,
  14,
  77,
  78,
  65,
  79,
  80,
  81,
  82,
  12,
  76,
  19,
  78,
  75,
  72,
  19,
 

In [ ]:
from functools import reduce

import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
import os

# ensures that we run only on cpu
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


class Model(tf.keras.Model):
    def __init__(self, vocab_size):
        """
        The Model class predicts the next words in a sequence.

        :param vocab_size: The number of unique words in the data
        """

        super(Model, self).__init__()

        # TODO: initialize emnbedding_size, batch_size, and any other hyperparameters

        self.vocab_size = vocab_size
        self.embedding_size = 30  # TODO
        self.batch_size = 300 # TODO

        self.optimizer = tf.keras.optimizers.Adam(1e-3) # learning rate is 1e-3
        self.loss = tf.keras.losses.sparse_categorical_crossentropy


        # TODO: initialize embeddings and forward pass weights (weights, biases)
        def make_vars(*dims, initializer=tf.random.normal):
          return tf.Variable(initializer(dims, stddev=.1))
        
        self.E = make_vars(self.vocab_size, self.embedding_size)   # E means embedding matrix 
        self.W1 = make_vars(2 * self.embedding_size, self.vocab_size) # because trigram has two inputs 
        self.b1 = make_vars(self.vocab_size)
        self.W2 = make_vars(self.vocab_size, self.vocab_size)   # weight
        self.b2 = make_vars(self.vocab_size)   # bias

    def call(self, inputs):
        """
        You must use an embedding layer as the first layer of your network
        (i.e. tf.nn.embedding_lookup)

        :param inputs: word ids of shape (batch_size, 2)
        :return: probabilities: The batch element probabilities as a tensor of shape (batch_size, vocab_size)
        """

        # TODO: Fill in
        embed_lookup_0 = tf.nn.embedding_lookup(self.E, inputs[:, 0])  # (inputs[:, 0], embed_size)
        embed_lookup_1 = tf.nn.embedding_lookup(self.E, inputs[:, 1])  # (inputs[:, 1], embed_size)
        embed_inputs = tf.concat([embed_lookup_0, embed_lookup_1], 1) # (inputs, 2 * embed_size)
        dense1 = tf.matmul(embed_inputs, self.W1) + self.b1
        # (inputs, 2*embed_size) * (2*embed_size, vocab_size) + vocab_size
        dense1_relu = tf.nn.relu(dense1)
        dense2 = tf.matmul(dense1_relu, self.W2) + self.b2
        # (inputs, vocab_size) * (vocab_size, vocab_size) + vocab_size

        prob = tf.nn.softmax(dense2)

        return prob


        

    def loss_function(self, probabilities, labels):
        """
        Calculates average cross entropy sequence to sequence loss of the prediction

        :param probabilities: a matrix of shape (batch_size, vocab_size)
        :return: the average loss of the model as a tensor of size 1
        """
        # TODO: Fill in
        # We recommend using tf.keras.losses.sparse_categorical_crossentropy

        return tf.reduce_mean(self.loss(labels, probabilities))  # syntax: y_true, y_predict 


def train(model, train_inputs, train_labels):
    """
    Runs through one epoch - all training examples.
    Remember to shuffle your inputs and labels - ensure that they are shuffled
    in the same order. Also you should batch your input and labels here.

    :param model: the initilized model to use for forward and backward pass
    :param train_input: train inputs (all inputs for training) of shape (num_inputs,2)
    :param train_input: train labels (all labels for training) of shape (num_inputs,)
    :return: None
    """

    # TODO Fill in
    
    # shuffle the inputs and labels
    random_index = tf.random.shuffle(np.array(range(train_inputs.shape[0])))
    train_inputs = tf.gather(train_inputs, random_index)
    train_labels = tf.gather(train_labels, random_index)



    # use batch_size to train the model 
    train_steps = int(train_inputs.shape[0] / model.batch_size)

    for i in range(train_steps):
      start = i * model.batch_size
      end = (i + 1) * model.batch_size
      if (i + 1) * model.batch_size > train_inputs.shape[0]:
        end = train_inputs.shape[0]
      X = train_inputs[start:end]
      Y = train_labels[start:end]
      
      with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = model.loss_function(y_pred, Y)

      gradients = tape.gradient(loss, model.trainable_variables)
      model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))



def test(model, test_inputs, test_labels):
    """
    Runs through all test examples. Test input should be batched here.

    :param model: the trained model to use for prediction
    :param test_input: train inputs (all inputs for testing) of shape (num_inputs,2)
    :param test_input: train labels (all labels for testing) of shape (num_inputs,)
    :returns: perplexity of the test set
    """

    # TODO: Fill in
    # NOTE: Ensure a correct perplexity formula (different from raw loss)

    train_steps = int(test_inputs.shape[0] / model.batch_size)

    total_loss = []
    for i in range(train_steps):
      start = i * model.batch_size
      end = (i + 1) * model.batch_size
      if (i + 1) * model.batch_size > test_inputs.shape[0]:
        end = test_inputs.shape[0]
      X = test_inputs[start:end]
      Y = test_labels[start:end]

      # average cross entropy loss for 1 batch (similar to 1 sentence)
      loss = tf.reduce_mean(model.loss_function(model(X), Y))
      total_loss.append(loss)

    perplexity = np.exp(np.mean(total_loss))

    return perplexity
      
    

    


def generate_sentence(word1, word2, length, vocab, model):
    """
    Given initial 2 words, print out predicted sentence of targeted length.

    :param word1: string, first word
    :param word2: string, second word
    :param length: int, desired sentence length
    :param vocab: dictionary, word to id mapping
    :param model: trained trigram model

    """

    # NOTE: This is a deterministic, argmax sentence generation

    reverse_vocab = {idx: word for word, idx in vocab.items()}
    output_string = np.zeros((1, length), dtype=np.int)
    output_string[:, :2] = vocab[word1], vocab[word2]

    for end in range(2, length):
        start = end - 2
        output_string[:, end] = np.argmax(model(output_string[:, start:end]), axis=1)
    text = [reverse_vocab[i] for i in list(output_string[0])]

    print(" ".join(text))


def main():
    # TODO: Pre-process and vectorize the data using get_data from preprocess
    train2id, test2id, word2id = get_data("train.txt", "test.txt")

    # TO-DO:  Separate your train and test data into inputs and labels
    first_word_train = np.array(train2id[0:len(train2id)-2]).reshape(-1,1)
    second_word_train = np.array(train2id[1:len(train2id)-1]).reshape(-1,1)
    inputs_train = np.concatenate((first_word_train, second_word_train), 1)
    labels_train = train2id[2:len(train2id)]

    first_word_test = np.array(test2id[0:len(test2id)-2]).reshape(-1,1)
    second_word_test = np.array(test2id[1:len(test2id)-1]).reshape(-1,1)
    inputs_test = np.concatenate((first_word_test, second_word_test), 1)
    labels_test = test2id[2:len(test2id)]

    # TODO: initialize model
    trigram = Model(len(word2id))

    # TODO: Set-up the training step
    train(trigram, inputs_train, labels_train)


    # TODO: Set up the testing steps
    perplexity = test(trigram, inputs_test, labels_test)


    # Print out perplexity
    print(perplexity)

    # BONUS: Try printing out sentences with different starting words

    


if __name__ == "__main__":
    main()



KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
import os

# ensures that we run only on cpu
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


class Model(tf.keras.Model):
    def __init__(self, vocab_size):
        """
        The Model class predicts the next words in a sequence.

        :param vocab_size: The number of unique words in the data
        """

        super(Model, self).__init__()

        # TODO: initialize embedding_size, batch_size, and any other hyperparameters

        self.vocab_size = vocab_size
        self.window_size = 20
        self.embedding_size = 30  # TODO
        self.batch_size = 100  # TODO

        self.rnn_size = 30   ## dimension of inner cell such as hidden state, output state, and cell state for exactly one LSTM block 

        # TODO: initialize embeddings and forward pass weights (weights, biases)
        # Note: You can now use tf.keras.layers!
        # - use tf.keras.layers.Dense for feed forward layers
        # - and use tf.keras.layers.GRU or tf.keras.layers.LSTM for your RNN


        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
        self.l = tf.keras.losses.sparse_categorical_crossentropy

        ## RNNs
        self.LSTM = tf.keras.layers.LSTM(self.rnn_size, return_sequences=True, return_state=True)

        # return_sequences = True: all hidden states (hidden state of all the time steps)
        # return_state = True: last hidden state + last cell state
        
        # Architecture 
        self.emb = tf.keras.layers.Embedding(self.vocab_size, self.embedding_size) 
        # The model will take as input an integer matrix of size (batch_size, window_size)
        # and the largest integer (i.e. word index) in the input should be no larger than vocab_size

        self.mlp1 = tf.keras.layers.Dense(100, activation = 'relu') 
        self.mlp2 = tf.keras.layers.Dense(self.vocab_size, activation = 'softmax')




    def call(self, inputs, initial_state):
        """
        - You must use an embedding layer as the first layer of your network
        (i.e. tf.nn.embedding_lookup)
        - You must use an LSTM or GRU as the next layer.

        :param inputs: word ids of shape (batch_size, window_size)
        :param initial_state: 2-d array of shape (batch_size, rnn_size) as a tensor
        :return: the batch element probabilities as a tensor, a final_state
        (NOTE 1: If you use an LSTM, the final_state will be the last two RNN outputs,
        NOTE 2: We only need to use the initial state during generation)
        using LSTM and only the probabilites as a tensor and a final_state as a tensor when using GRU
        """

        # TODO: Fill in
        emb_inputs = self.emb(inputs) # embedding lookup
                                      # (batch_size, window_size, embedding_size)

        print(f'this is the shape of emb_inputs: {emb_inputs.shape}')  # (100, 20, 30)


        whole_seq_output, final_hidden_state, final_cell_state = self.LSTM(emb_inputs, initial_state = initial_state)
        # inputs should be: (batch_size, timesteps(window_size), feature(embedding_size))
        # whole_seq_output: (batch_size, timesteps(window_size), unit/rnn_size)
        # final_hidden_state, final_cell_state: (batch_size, unit/rnn_size): the final embedding for each batch_size element 
                                                                           # after going through the lstm block 20 times


        # if the window_size is 20, the model can only model relationship within these 20 words
        # there will be 20 lstm blocks, each time, it take one word as input
        # basically trying to use first 19 words to predict the last word if the window_size is 20 


        print(f'this is the shape of whole_seq_output: {whole_seq_output.shape}')   # (100, 20, 30)
        print(f'this is the shape of final_hidden_state: {final_hidden_state.shape}')  # (100, 30)
        print(f'this is the shape of final_cell_state: {final_cell_state.shape}')      # (100, 30)


        dense = self.mlp1(whole_seq_output)
        prob = self.mlp2(dense)

        return prob, (final_hidden_state, final_cell_state)

    def loss(self, probabilities, labels):
        """
        Calculates average cross entropy sequence to sequence loss of the prediction

        :param probabilities: a matrix of shape (batch_size, window_size, vocab_size) as a tensor
        :param labels: matrix of shape (batch_size, window_size) containing the labels
        :return: the average loss of the model as a tensor of size 1
        """

        # TODO: Fill in
        # We recommend using tf.keras.losses.sparse_categorical_crossentropy

        return tf.reduce_mean(self.l(labels, probabilities))


def train(model, train_inputs, train_labels):
    """
    Runs through one epoch - all training examples (remember to batch!)
    Here you will also want to reshape your inputs and labels so that they match
    the inputs and labels shapes passed in the call and loss functions respectively.

    :param model: the initilized model to use for forward and backward pass
    :param train_inputs: train inputs (all inputs for training) of shape (num_inputs,)
    :param train_labels: train labels (all labels for training) of shape (num_labels,)
    :return: None
    """
    # TODO: Fill in


    ## reshape

    # print(f'this is train_inputs.shape: {np.array(train_inputs).shape}')  # (1465613,)
    num = len(train_inputs) // model.window_size
    train_inputs = np.array(train_inputs[0:num*model.window_size]).reshape(-1, model.window_size)
    train_labels = np.array(train_labels[0:num*model.window_size]).reshape(-1, model.window_size)
    # print(f'this is train_inputs.shape: {train_inputs.shape}')          # (73280, 20)




    # shuffle the inputs and labels
    random_index = tf.random.shuffle(np.array(range(train_inputs.shape[0])))
    train_inputs = tf.gather(train_inputs, random_index)
    train_labels = tf.gather(train_labels, random_index)



    # use batch_size to train the model 
    train_steps = int(train_inputs.shape[0] / model.batch_size)

    for i in range(train_steps):
      start = i * model.batch_size
      end = (i + 1) * model.batch_size
      if (i + 1) * model.batch_size > train_inputs.shape[0]:
        end = train_inputs.shape[0]
      X = train_inputs[start:end]
      Y = train_labels[start:end]
      
      with tf.GradientTape() as tape:
        prob, final_state = model.call(X, initial_state = None)
        total_loss = model.loss(prob, Y)
      
      gradients = tape.gradient(total_loss, model.trainable_variables)
      model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))


def test(model, test_inputs, test_labels):
    """
    Runs through one epoch - all testing examples (remember to batch!)
    Here you will also want to reshape your inputs and labels so that they match
    the inputs and labels shapes passed in the call and loss functions respectively.

    :param model: the trained model to use for prediction
    :param test_inputs: train inputs (all inputs for testing) of shape (num_inputs,)
    :param test_labels: train labels (all labels for testing) of shape (num_labels,)
    :returns: perplexity of the test set
    """

    # TODO: Fill in
    # NOTE: Ensure a correct perplexity formula (different from raw loss)


    # reshape 
    num = len(test_inputs) // model.window_size
    test_inputs = np.array(test_inputs[0:num*model.window_size]).reshape(-1, model.window_size)
    test_labels = np.array(test_labels[0:num*model.window_size]).reshape(-1, model.window_size)


    train_steps = int(test_inputs.shape[0] / model.batch_size)

    
    total_loss = []
    for i in range(train_steps):
      start = i * model.batch_size
      end = (i + 1) * model.batch_size
      if (i + 1) * model.batch_size > test_inputs.shape[0]:
        end = test_inputs.shape[0]
      X = test_inputs[start:end]
      Y = test_labels[start:end]

      # average cross entropy loss for 1 batch (similar to 1 sentence)
      
      prob, final_state = model.call(X, initial_state = None)
      batch_loss = tf.reduce_mean(model.loss(prob, Y))
      total_loss.append(batch_loss)

    perplexity = np.exp(np.mean(total_loss))

    return perplexity

def generate_sentence(word1, length, vocab, model, sample_n=10):
    """
    Takes a model, vocab, selects from the most likely next word from the model's distribution

    :param model: trained RNN model
    :param vocab: dictionary, word to id mapping
    :return: None
    """

    # NOTE: Feel free to play around with different sample_n values

    reverse_vocab = {idx: word for word, idx in vocab.items()}
    previous_state = None

    first_string = word1
    first_word_index = vocab[word1]
    next_input = [[first_word_index]]
    text = [first_string]

    for i in range(length):
        logits, previous_state = model.call(next_input, previous_state)
        logits = np.array(logits[0, 0, :])
        top_n = np.argsort(logits)[-sample_n:]
        n_logits = np.exp(logits[top_n]) / np.exp(logits[top_n]).sum()
        out_index = np.random.choice(top_n, p=n_logits)

        text.append(reverse_vocab[out_index])
        next_input = [[out_index]]

    print(" ".join(text))


def main():
    # TODO: Pre-process and vectorize the data
    # HINT: Please note that you are predicting the next word at each timestep,
    # so you want to remove the last element from train_x and test_x.
    # You also need to drop the first element from train_y and test_y.
    # If you don't do this, you will see impossibly small perplexities.

    # TODO: Separate your train and test data into inputs and labels
    train_data, test_data, word2id = get_data("train.txt", "test.txt")

    inputs_train = train_data[0:len(train_data)-1]
    labels_train = train_data[1:len(train_data)]
    inputs_test = test_data[0:len(test_data)-1]
    labels_test = test_data[1:len(test_data)]
    vocab_size = len(word2id)

    # TODO: initialize model and tensorflow variables
    LSTM = Model(vocab_size)

    # TODO: Set-up the training step
    train(LSTM, inputs_train, labels_train)

    # TODO: Set up the testing steps
    perplexity = test(LSTM, inputs_test, labels_test)

    # Print out perplexity
    print(perplexity)

    # BONUS: Try printing out various sentences with different start words and sample_n parameters


if __name__ == "__main__":
    main()

this is the shape of emb_inputs: (100, 20, 30)
this is the shape of whole_seq_output: (100, 20, 30)
this is the shape of final_hidden_state: (100, 30)
this is the shape of final_cell_state: (100, 30)
this is the shape of emb_inputs: (100, 20, 30)
this is the shape of whole_seq_output: (100, 20, 30)
this is the shape of final_hidden_state: (100, 30)
this is the shape of final_cell_state: (100, 30)
this is the shape of emb_inputs: (100, 20, 30)
this is the shape of whole_seq_output: (100, 20, 30)
this is the shape of final_hidden_state: (100, 30)
this is the shape of final_cell_state: (100, 30)
this is the shape of emb_inputs: (100, 20, 30)
this is the shape of whole_seq_output: (100, 20, 30)
this is the shape of final_hidden_state: (100, 30)
this is the shape of final_cell_state: (100, 30)
this is the shape of emb_inputs: (100, 20, 30)
this is the shape of whole_seq_output: (100, 20, 30)
this is the shape of final_hidden_state: (100, 30)
this is the shape of final_cell_state: (100, 30)


KeyboardInterrupt: ignored